## Imports

In [ ]:
import numpy as np
import torch
from torchvision import transforms
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm, trange
import matplotlib.pyplot as plt

# from custom files
from dataset import CompCarsImageFolder, WrapperDataset
from models import ResNet, resnet_cfg
from models import train, validate
from utils import *

## Configuration

In [ ]:
###### Set root to the image folder of CompCars dataset ######

#TODO: ADAPT TO YOUR FOLDER STRUCTURE
root = '/Volumes/EDO/NNDL/CompCars dataset/data/image/'

#############################################################

### Hyperparam configuration # TODO: TUNE
params = {                  ## Training Params (inspired from original resnet paper: https://arxiv.org/pdf/1512.03385)
    'epoch_num': 50,        # number of epochs
    'lr': 1e-1,             # (initial) Learning Rate
    'weight_decay': 1e-4,   # L2 Penalty
    'batch_size': 64,       # batch size (depends on hardware)
    'momentum': 0.9,
    
    'hierarchy': 0,         # Choose 0 for manufacturer classification, 1 for model classification
    'val_split': 10000,     # (float) Fraction of validation holdout / (int) Absolute number of data points in holdout
    
    'resnet': resnet_cfg['resnet18']  # Resnet model used
}

### Device
if torch.cuda.is_available():
    params["device"] = torch.device("cuda")   # option for NVIDIA GPUs
elif torch.backends.mps.is_available():
    params["device"] = torch.device("mps")    # option for Mac M-series chips (GPUs)
else:
    params["device"] = torch.device("cpu")    # default option if none of the above devices are available

print("Device: {}".format(params["device"]))

# Data Pipeline

## Read total Dataset

In [ ]:
# hierarchy=0 -> manufacturer classification; hierarchy=1 -> model classification
total_set = CompCarsImageFolder(root, hierarchy=params['hierarchy'])
print(total_set.classes)
print(len(total_set.classes))

## Raw Data Visualization

In [ ]:
# Visualization of the dataset
label_dict = {y: x for x, y in total_set.class_to_idx.items()}

num_images_to_show = 15
data_idx = np.random.randint(0, high=len(total_set), size=num_images_to_show)
num_cols = 5
num_rows = num_images_to_show // num_cols

fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, num_rows * 3))
axes = axes.flatten()
for i in range(num_images_to_show):
    image, label = total_set[data_idx[i]]
    npimg = np.array(image)
    axes[i].imshow(npimg)
    axes[i].set_title(label_dict[label])
    axes[i].axis('off')
    print(f"Image shape: {image.size}")
plt.tight_layout()
plt.show()

## Split in training and validation data

In [ ]:
datasets = train_val_dataset(total_set, val_split=params['val_split'])

## Compute normalization statistics

In [ ]:
'''
NOTE: This cell takes some time. Could be accelerated by:
    1. Using dataloader (vectorized batches)
    2. Resize images before computing statistics
'''
# Default values (taken from the results from this cell)
mean, std = [0.483, 0.471, 0.463], [0.297, 0.296, 0.302]

# Compute mean and std for training dataset
# train_mean, train_std = compute_mean_std_from_dataset(datasets['train'])
# print(f"Training dataset mean: {train_mean}")
# print(f"Training dataset std: {train_std}")
train_mean, train_std = mean, std

# Compute mean and std for validation dataset
# val_mean, val_std = compute_mean_std_from_dataset(datasets['val'])
# print(f"Validation dataset mean: {val_mean}")
# print(f"Validation dataset std: {val_std}")
val_mean, val_std = mean, std

## Transform data - Prepare DataLoaders

In [ ]:
# Apply transformation
########################## Transforms ############################
# TODO: Adapt transforms to our data set
# TODO: maybe use v2 transforms: https://pytorch.org/vision/stable/transforms.html

# inspired from https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
data_transforms = { # TODO: TUNE
    'train': transforms.Compose([
        transforms.RandomChoice([
            transforms.Resize(256),
            transforms.Resize(224),
            transforms.Resize(320)
        ]),
        transforms.RandomHorizontalFlip(),
        transforms.RandomCrop(224),
        transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4),
        transforms.ToTensor(),
        transforms.Normalize(train_mean, train_std)
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(val_mean, val_std)
    ])
}
##################################################################

wrapped_datasets = {
    'train': WrapperDataset(datasets['train'], transform=data_transforms['train']),
    'val': WrapperDataset(datasets['val'], transform=data_transforms['val'])
}

dataloaders = {
    'train': DataLoader(wrapped_datasets['train'], batch_size=params['batch_size'], shuffle=True, num_workers=4),
    'val': DataLoader(wrapped_datasets['val'], batch_size=params['batch_size'], shuffle=False, num_workers=4)
}


print(f"Total dataset size: {len(total_set)}")
print(f"Training dataset size: {len(datasets['train'])}")
print(f"Validation dataset size: {len(datasets['val'])}")

x, y = next(iter(dataloaders['train']))
print(f"Batch of training images shape: {x.shape}")
print(f"Batch of training labels shape: {y.shape}")

x, y = next(iter(dataloaders['val']))
print(f"Batch of validation images shape: {x.shape}")
print(f"Batch of validation labels shape: {y.shape}")

# Training

## Set-Up

In [ ]:
# Set up resnet model
resnet = ResNet(params['resnet']['block'], params['resnet']['layers'], 
                len(total_set.classes)).to(params['device'])

# Loss and Optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD( #SGD used in original resnet paper # TODO: TUNE
    resnet.parameters(), 
    lr=params['lr'], 
    weight_decay=params['weight_decay'], 
    momentum=params['momentum']
)

# LR scheduler (using Top-1-Accuracy as Validation metric)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, min_lr=1e-5, patience=1, threshold=1e-2) # TODO: TUNE

# Save performance metrics
train_losses, validation_losses, train_acc, validation_acc, train_top5_acc, validation_top5_acc = list(), list(), list(), list(), list(), list()

## Training Loop

To start from checkpoint, set `START_FROM_CHECKPOINT=True`.

In [ ]:
CHECKPOINT_PATH = './training_checkpoints/checkpoint.pth'
START_FROM_CHECKPOINT = False   # set to TRUE to start from checkpoint
start_epoch = 0

if START_FROM_CHECKPOINT:
    checkpoint = torch.load(CHECKPOINT_PATH)
    start_epoch = checkpoint['epoch']
    resnet.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    train_losses = checkpoint['train_losses']
    train_acc = checkpoint['train_acc']
    train_top5_acc = checkpoint['train_top5_acc']
    validation_losses = checkpoint['validation_losses']
    validation_acc = checkpoint['validation_acc']
    validation_top5_acc = checkpoint['validation_top5_acc']

# Just some fancy progress bars # FIXME: inside epoch progress bar not working reliably for me
pbar_epoch = trange(start_epoch, params["epoch_num"], initial=start_epoch, desc="Training", position=0)
pbar_inside_epoch = trange(0, (len(dataloaders['train'])+len(dataloaders['val'])), desc="Training and validation per epoch", position=1, leave=False)

# Stop the training phase in case there is no improvement
# early_stopper = EarlyStopper(patience=10, min_delta=0.1)

for epoch in pbar_epoch:
    pbar_inside_epoch.reset()

    # Training
    train_results = train(dataloaders['train'], resnet, epoch, criterion, optimizer, params["device"], pbar=pbar_inside_epoch)
    train_losses.append(train_results[0])
    train_acc.append(1 - train_results[1])                 # saving acc error
    train_top5_acc.append(1 - train_results[2])

    # Validation
    validation_results = validate(dataloaders['val'], resnet, epoch, criterion, params["device"], pbar=pbar_inside_epoch)
    validation_losses.append(validation_results[0])
    validation_acc.append(1 - validation_results[1])       # saving acc error
    validation_top5_acc.append(1 - validation_results[2])

    # Scheduler
    scheduler.step(validation_results[1])   # ReduceLROnPlateau scheduler (reduce LR by 10 when top-1-accuracy does not improve)
    print(f"\nCurrent Learning Rate: {scheduler._last_lr}\n")

    # Checkpoint
    torch.save({
        'epoch' : epoch + 1,
        'model_state_dict' : resnet.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict' : scheduler.state_dict(),
        'train_losses': train_losses,
        'train_acc': train_acc,
        'train_top5_acc': train_top5_acc,
        'validation_losses': validation_losses,
        'validation_acc': validation_acc,
        'validation_top5_acc': validation_top5_acc,
    }, CHECKPOINT_PATH)

    # Comment on the following lines if you don't want to stop early in case of no improvement
    # if early_stopper.early_stop(validation_results[0]):
    #     params['epoch_num'] = epoch
    #     print("\n\nEarly stopping...")
    #     break

pbar_inside_epoch.close()

## Plot losses

In [ ]:
# Plotting the performance of the model in the training and validation phase

params["epoch_num"] = 39

plots = [
    (np.arange(0, params["epoch_num"], 1), train_losses, "Train Loss"),
    (np.arange(0, params["epoch_num"], 1), validation_losses, "Validation Loss")
]

show_plot(plots, "Model Loss for Epoch", "Epoch", "Loss")

plots = [
    (np.arange(0, params["epoch_num"], 1), train_acc, "Train Top-1-Error"),
    (np.arange(0, params["epoch_num"], 1), validation_acc, "Validation Top-1-Error"),
    (np.arange(0, params["epoch_num"], 1), train_top5_acc, "Train Top-5-Error"),
    (np.arange(0, params["epoch_num"], 1), validation_top5_acc, "Validation Top-5-Error")
]

show_plot(plots, "Model Classification Error for Epoch", "Epoch", "Error Rate")

## Save Model

In [ ]:
MODEL_PATH = './trained_models/resnet18_weights_cars_maker.pth' # TODO: adapt
torch.save({
    'model_state_dict' : resnet.state_dict(),
    'train_losses': train_losses,
    'train_acc': train_acc,
    'train_top5_acc': train_top5_acc,
    'validation_losses': validation_losses,
    'validation_acc': validation_acc,
    'validation_top5_acc': validation_top5_acc,
    'resnet': params['resnet'],
    }, MODEL_PATH)

# Load and Evaluate Saved Model

## Load

In [ ]:
MODEL_PATH = './trained_models/resnet18_weights_cars_maker.pth'
saved_model = torch.load(MODEL_PATH)

resnet = ResNet(saved_model['resnet']['block'], saved_model['resnet']['layers'], 
                len(total_set.classes)).to(params['device'])

resnet.load_state_dict(saved_model['model_state_dict'])
train_losses = saved_model['train_losses']
train_acc = saved_model['train_acc']
train_top5_acc = saved_model['train_top5_acc']
validation_losses = saved_model['validation_losses']
validation_acc = saved_model['validation_acc']
validation_top5_acc = saved_model['vaidation_top5_acc']

## Plot performance data

In [ ]:
# Plotting the performance of the model in the training and validation phase

plots = [
    (np.arange(0, params["epoch_num"], 1), train_losses, "Train Loss"),
    (np.arange(0, params["epoch_num"], 1), validation_losses, "Validation Loss")
]

show_plot(plots, "Model Loss for Epoch", "Epoch", "Loss")

plots = [
    (np.arange(0, params["epoch_num"], 1), train_acc, "Train Top-1-Error"),
    (np.arange(0, params["epoch_num"], 1), validation_acc, "Validation Top-1-Error"),
    (np.arange(0, params["epoch_num"], 1), train_top5_acc, "Train Top-5-Error"),
    (np.arange(0, params["epoch_num"], 1), validation_top5_acc, "Validation Top-5-Error")
]

show_plot(plots, "Model Classification Error for Epoch", "Epoch", "Error Rate")